In [1]:
# This snippet of code properly adds the working source root path to python's path
# so you no longer have to install spykshrk through setuptools
import sys, os
root_depth = 2
notebook_dir = globals()['_dh'][0]
root_path = os.path.abspath(os.path.join(notebook_dir, '../'*root_depth))
# Add to python's path
try:
    while True:
        sys.path.remove(root_path)
except ValueError:
    # no more root paths
    pass
sys.path.append(root_path)
# Alternatively set root path as current working directory
#os.chdir(root_path)

In [2]:
import enum
import collections
import copy
import pprint
import numpy as np
import IPython
import IPython.lib

plain_formatter = get_ipython().display_formatter.formatters['text/plain']
plain_formatter.max_width=160
plain_formatter.max_seq_length=20

np.set_printoptions(precision=4, linewidth=80, threshold=20, edgeitems=5)


In [3]:
plain_formatter.__dir__

In [17]:
class AttrDict(collections.UserDict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __setattr__(self, key, value):
        super().__setattr__(key, value)
        if hasattr(self, 'data') and (key is not 'data'):

            self.data[key] = value

    def __getattribute__(self, item):
        try:
            return super().__getattribute__(item)
        except AttributeError:
            try:
                return self[item]
            except KeyError:
                raise AttributeError(item)

    def __getitem__(self, key):
        if hasattr(self, 'data'):
            return super().__getitem__(key)

    def __getstate__(self):
        return self.data

    def __setstate__(self, state):
        self.data = state
    
    def _repr_pretty_(self, pp, cycle):
        if cycle:
            pp.text(f'{type(self).__name__}(...)')
        else:
            with pp.group(1, '{', '}'):
            #with pp.group(len(type(self).__name__)+2, f'{type(self).__name__}({{', '})'):
                for idx, (key, val) in enumerate(self.data.items()):
                    if idx is not 0:
                        pp.text(',')
                        pp.breakable()
                    pp.text(str(key) + ': ')
                    with pp.indent(len(str(key)) + 2):
                        if (isinstance(val, collections.abc.Sequence) and not
                            isinstance(val, (str, bytes, bytearray, memoryview))):
                            with pp.group(1, '[', ']'):
                                if len(val) > pp.max_seq_length:
                                    short_val = val[0:int(pp.max_seq_length/2)] + val[-int(pp.max_seq_length/2):]
                                else:
                                    short_val = val
                                for ent_ii, ent in enumerate(short_val):
                                    if (len(val) > pp.max_seq_length) and (ent_ii == int(pp.max_seq_length/2)):
                                        if pp.output_width + 6 > pp.max_width:
                                            pp.breakable()
                                        pp.text(', ... ')
                                        
                                    if (isinstance(ent, collections.abc.Collection) and not 
                                        isinstance(ent, (str, bytes, bytearray, memoryview))):
                                        if ent_ii is not 0:
                                            pp.text(',')
                                            pp.breakable()
                                        pp.pretty(ent)
                                        was_last_coll = True
                                    else:
                                        if ent_ii is not 0:
                                            pp.text(', ')
                                            if pp.output_width + len(str(ent)) > pp.max_width:
                                                pp.breakable()
                                            elif was_last_coll:
                                                pp.breakable()
                                        was_last_coll = False
                                        pp.pretty(ent)
                        elif isinstance(val, (str, bytes, bytearray, memoryview)):
                            pp.pretty(val)
                        elif isinstance(val, (np.ndarray, np.generic)):
                            #pp.output_width += np.get_printoptions()['linewidth']
                            pp.pretty(val)
                        else:
                            pp.pretty(val)
            #pp.breakable()
                
    def __repr__(self):
        return f"{type(self).__name__}({self.data})"

In [5]:
l = [10,20,30,40,50]
l+[60]
l

In [15]:
collections.abc.Collection?

In [18]:
AttrDict({'int': 1000,
          'float': 12.34,
          'list': list(range(50)),
          'str': 'test_string',
          'long str': ', '.join(['a']*100),
          'list str': ['test'],
          'list str 2': [', '.join(['a']*50)]*2,
          'list str 3': ['a']*100,
          'list_str 4': ['list']*100,
          'list nest': [['first']*4, ['second']*4, ['third']*4, ['fourth']*4],
          'np array': np.zeros(100),
          'list np array': [np.zeros(100)]*3,
          'AttrDict': AttrDict({'int':200, 'list': list(range(50)), 'list 2': list(range(50))}),
          'AttrDict 2': AttrDict({'int':200, 'list': list(range(50))})})

In [150]:
test = 'a b c' ' d e f'
test

In [23]:
ad = AttrDict({i:i for i in range(10)})
ad[20] = AttrDict({i:i for i in range(10)})
ad[20][20] = ['a']*40
ad[20][20][25] = AttrDict({i:i for i in range(10)})
ad

In [179]:
d = {i:i for i in range(20)}
d[10] = ['a'] * 20
d[10][2] = {i:i for i in range(20)}
pprint.pprint(d, compact=True)

In [118]:
pprint.pprint({x:x for x in range(40)}, compact=True)

In [12]:
ud = collections.UserDict({'key': 'one'})
ud

In [9]:
copy.deepcopy(ad)

In [4]:
class AttrDictEnum(collections.UserDict):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __getitem__(self, item):
        try:
            return super().__getitem__(item)
        except KeyError as kerr:
            retval = []
            keylist = []
            for key, val in self.__dict__['data'].items():
                try:
                    if item == key.name:
                        keylist = [key]
                        retval.append(val)
                except AttributeError:
                    if item == key:
                        keylist = [key]
                        retval.append(val)
            if len(retval) > 1:
                raise KeyError('Found multiple keys, possible overlapping enums: ' + keylist)
            elif len(retval) == 0:
                raise KeyError(item)
            return retval[0]

    def __setitem__(self, key, value):
        super().__setitem__(key, value)

    def __getattribute__(self, item):
        try:
            return super().__getattribute__(item)
        except AttributeError as attr_err:
            try:
                return self.__getitem__(item)
            except KeyError as kerr:
                raise AttributeError(*kerr.args)

    def __getstate__(self):
        return self.data

    def __setstate__(self, state):
        self.data = state

    def __repr__(self):
        return f"{type(self).__name__}({self.data})"


In [4]:
class Test(enum.Enum):
    one = 1
    two = 2
    
d = AttrDictEnum({Test.one: 111})

In [5]:
d.data

In [6]:
Test.one.__str__()

In [7]:
d[Test.two] = 2
d.two

In [8]:
d.items()

In [9]:
from typing import ItemsView

dv = ItemsView(d)
dv

In [10]:
d.__dict__

In [11]:
dv

In [12]:
dd = {'one': 1, 'one':2}
it = dd.items()
it